In [264]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4
n_runs = 300
seq_length = [180, 360, 540, 720, 900]


def read_sequences():
    run_list_mix = []
    run_list_ordered = []
    labels_reordered = []
    labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
    labels = labels.drop(columns="Unnamed: 0")
    labels = np.array(labels)
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    for index, length in enumerate(seq_length):
        run_list_ordered.append([])
        for run_index,run in enumerate(run_list_mix): 
            if len(run) == length:
                run_list_ordered[index].append(run)    
                labels_reordered.append(labels[run_index])
    return run_list_ordered, labels_reordered
    
    
def stadard_sequences(sequences):
    for index, seqs in enumerate(sequences):
        for i, seq in enumerate(seqs):
            seqs[i] = MinMaxScaler(feature_range=[0,1]).fit_transform(seq)
    return sequences                    


trainX_sequences, labels_ordered = read_sequences()
trainX_sequences = stadard_sequences(trainX_sequences)
for i, leng in enumerate(seq_length):
    trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))

# 
# def train_generator():
#     index = 0
#     while True:
#         x_train = trainX_sequences[index]
#         index += 1
#         yield (x_train, x_train) 


In [265]:
from keras import Sequential
from keras.layers import Conv2DTranspose, LSTM, RepeatVector
from keras.losses import mse
import tensorflow as tf

intermediate_dimensions = 50
latent_dim = 20
filters = 50

def reshape(x):
    x_shape = K.shape(x)
    new_x_shape = K.concatenate([x_shape[:-2], [x_shape[-2] * x_shape[-1]]])    
    print(new_x_shape)
    return K.reshape(x, new_x_shape)

def create_ae(verbose):
    filters = 50
    input = Input(shape=(None, n_features))
    x = input
    
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, activation='relu', padding='same')(x)
    x = Conv1D(filters=filters, kernel_size=2, activation='relu', padding='same')(x)
    shape = K.int_shape(x)
    enc = Dense(latent_dim)(x)
    
    encoder = Model(input, enc)
    if verbose:
        encoder.summary()
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_input')
    x = Dense(intermediate_dimensions, name='Dense_after_enc', activation='relu')(latent_inputs)
    #x = Reshape(shape=(shape[-2],shape[-1]))(x)
    output = Dense(n_features, name='output')(x)
    
    decoder = Model(latent_inputs, output)
    if verbose:
        decoder.summary()
    
    output = decoder(encoder.output)
    vae = Model(input, output)
    
    vae.compile(optimizer='adam', loss='mse')
    
    return vae, encoder, decoder

vae, enc, dec = create_ae(verbose=True)


Epoch 1/50


32/74 [===========>..................] - ETA: 10s - loss: 0.3351

64/74 [========================>.....] - ETA: 1s - loss: 0.3197 

74/74 [==============================] - 8s 106ms/step - loss: 0.3138


Epoch 2/50
32/74 [===========>..................] - ETA: 0s - loss: 0.2504

...

32/54 [================>.............] - ETA: 0s - loss: 4.5673e-05

54/54 [==============================] - 1s 11ms/step - loss: 4.6213e-05


Epoch 50/50


32/54 [================>.............] - ETA: 0s - loss: 4.6703e-05

54/54 [==============================] - 1s 11ms/step - loss: 4.4995e-05


In [266]:
def train():
    for batch in trainX_sequences:
        vae.fit(batch, batch, epochs=50)


train()


Epoch 1/50


32/74 [===========>..................] - ETA: 10s - loss: 0.3351

64/74 [========================>.....] - ETA: 1s - loss: 0.3197 

74/74 [==============================] - 8s 106ms/step - loss: 0.3138


Epoch 2/50
32/74 [===========>..................] - ETA: 0s - loss: 0.2504

...

32/54 [================>.............] - ETA: 0s - loss: 4.5673e-05

54/54 [==============================] - 1s 11ms/step - loss: 4.6213e-05


Epoch 50/50


32/54 [================>.............] - ETA: 0s - loss: 4.6703e-05

54/54 [==============================] - 1s 11ms/step - loss: 4.4995e-05


In [267]:
from functools import reduce

from sklearn.decomposition import PCA

#masks = [return_mask(num, labels)[:, 0] for num in range(0, 9)]
encodings = []
for i,seqs in enumerate(trainX_sequences):   
    encodings.append(np.mean(enc.predict(seqs), axis=1))
#encodings = reduce(lambda x : x, encodings)
encodings_squeezed = []
for i in encodings:
    for j in i:
        encodings_squeezed.append(j)
encodings = np.array(encodings_squeezed)


In [268]:
def return_mask(num, labels):
    arg = np.squeeze(np.argwhere(labels == num))
    return arg
    
    
masks = [return_mask(num, np.array(labels_ordered))[:, 0] for num in range(0, 9)]


[0.61205614 0.2710784  0.06235165]


AttributeError: 'NoneType' object has no attribute 'sqrt'

In [273]:

def plot_pca(title): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o','o','o','o','^','^','^','^','^','^']
    # for index, mask in enumerate(masks):
    #     ax.scatter(principalComponents[:,0][mask], principalComponents[:,1][mask], 
    #                 principalComponents[:,2][mask], marker=markers[index])
    # 
    for mask in masks:
        plt.scatter(x=principalComponents[:,0][mask], y=principalComponents[:,1][mask], alpha=0.5)

    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    
    
scaler = StandardScaler()
enc_input = scaler.fit_transform(encodings) 
pca = PCA(3)
principalComponents = pca.fit_transform(enc_input)
print(pca.explained_variance_ratio_)
plot_pca('Sequences')
# plt.show()



plt.show()

[0.61205614 0.2710784  0.06235165]


AttributeError: 'NoneType' object has no attribute 'sqrt'